# SerpAPI

[SerpAPI](https://serpapi.com/) allows you to integrate search engine results into your LLM apps

This guide provides a quick overview for getting started with the SerpAPI [tool](/docs/integrations/tools/). For detailed documentation of all `SerpAPI` features and configurations head to the [API reference](https://api.js.langchain.com/classes/_langchain_community.tools_serpapi.SerpAPI.html).

## Overview

### Integration details

| Class | Package | [PY support](https://python.langchain.com/docs/integrations/tools/serpapi/) | Package latest |
| :--- | :--- | :---: | :---: |
| [SerpAPI](https://api.js.langchain.com/classes/_langchain_community.tools_serpapi.SerpAPI.html) | [`@langchain/community`](https://www.npmjs.com/package/@langchain/community) | ✅ |  ![NPM - Version](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

## Setup

The integration lives in the `@langchain/community` package, which you can install as shown below:

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/core
</Npm2Yarn>
```

### Credentials

Set up an API key [here](https://serpapi.com/) and set it as an environment variable named `SERPAPI_API_KEY`.

```typescript
process.env.SERPAPI_API_KEY = "YOUR_API_KEY"
```

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability:

```typescript
process.env.LANGSMITH_TRACING="true"
process.env.LANGSMITH_API_KEY="your-api-key"
```

## Instantiation

You can import and instantiate an instance of the `SerpAPI` tool like this:

In [1]:
import { SerpAPI } from "@langchain/community/tools/serpapi";

const tool = new SerpAPI();

## Invocation

### [Invoke directly with args](/docs/concepts/#invoke-with-just-the-arguments)

You can invoke the tool directly like this:

In [2]:
await tool.invoke({
  input: "what is the current weather in SF?"
});

{"type":"weather_result","temperature":"63","unit":"Fahrenheit","precipitation":"3%","humidity":"91%","wind":"5 mph","location":"San Francisco, CA","date":"Sunday 9:00 AM","weather":"Mostly cloudy"}


### [Invoke with ToolCall](/docs/concepts/#invoke-with-toolcall)

We can also invoke the tool with a model-generated `ToolCall`, in which case a `ToolMessage` will be returned:

In [3]:
// This is usually generated by a model, but we'll create a tool call directly for demo purposes.
const modelGeneratedToolCall = {
  args: {
    input: "what is the current weather in SF?"
  },
  id: "1",
  name: tool.name,
  type: "tool_call",
}

await tool.invoke(modelGeneratedToolCall)

ToolMessage {
  "content": "{\"type\":\"weather_result\",\"temperature\":\"63\",\"unit\":\"Fahrenheit\",\"precipitation\":\"3%\",\"humidity\":\"91%\",\"wind\":\"5 mph\",\"location\":\"San Francisco, CA\",\"date\":\"Sunday 9:00 AM\",\"weather\":\"Mostly cloudy\"}",
  "name": "search",
  "additional_kwargs": {},
  "response_metadata": {},
  "tool_call_id": "1"
}


## Chaining

We can use our tool in a chain by first binding it to a [tool-calling model](/docs/how_to/tool_calling/) and then calling it:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```


In [4]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai"

const llm = new ChatOpenAI({
  model: "gpt-4o-mini",
  temperature: 0,
})

In [6]:
import { HumanMessage } from "@langchain/core/messages";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableLambda } from "@langchain/core/runnables";

const prompt = ChatPromptTemplate.fromMessages(
  [
    ["system", "You are a helpful assistant."],
    ["placeholder", "{messages}"],
  ]
)

const llmWithTools = llm.bindTools([tool]);

const chain = prompt.pipe(llmWithTools);

const toolChain = RunnableLambda.from(
  async (userInput: string, config) => {
    const humanMessage = new HumanMessage(userInput,);
    const aiMsg = await chain.invoke({
      messages: [new HumanMessage(userInput)],
    }, config);
    const toolMsgs = await tool.batch(aiMsg.tool_calls, config);
    return chain.invoke({
      messages: [humanMessage, aiMsg, ...toolMsgs],
    }, config);
  }
);

const toolChainResult = await toolChain.invoke("what is the current weather in sf?");

In [7]:
const { tool_calls, content } = toolChainResult;

console.log("AIMessage", JSON.stringify({
  tool_calls,
  content,
}, null, 2));

AIMessage {
  "tool_calls": [],
  "content": "The current weather in San Francisco is mostly cloudy, with a temperature of 64°F. The humidity is at 90%, there is a 3% chance of precipitation, and the wind is blowing at 5 mph."
}


## Agents

For guides on how to use LangChain tools in agents, see the [LangGraph.js](https://langchain-ai.github.io/langgraphjs/) docs.

## API reference

For detailed documentation of all `SerpAPI` features and configurations head to the API reference: https://api.js.langchain.com/classes/_langchain_community.tools_serpapi.SerpAPI.html